# Исследование надежности заёмщиков

### Шаг 1. Импорт библиотек и получение первичной информации

In [1]:
import pandas as pd
credit = pd.read_csv('/datasets/data.csv') #cоздаем переменную для нашего датафрейма

FileNotFoundError: [Errno 2] File /datasets/data.csv does not exist: '/datasets/data.csv'

In [2]:
credit.info() #получаем информацию о структуре таблицы
#просмотрим количество уникальных значений по всем столбцам таблицы
#credit['children'].value_counts() - отрицательное количество детей и аномальное количество у почти 80 человек
#credit['days_employed'].value_counts() #- есть отрицательные и пропущенные значения
#credit['dob_years'].value_counts() - все хорошо
#credit['education'].value_counts() - есть проблемы с регистром, позже решим это
#credit['education_id'].value_counts() - все хорошо
#credit['family_status'].value_counts() - есть проблема с регистром
#credit['family_status_id'].value_counts() - все хорошо
#credit['gender'].value_counts() - есть пропущенное значение 
#credit['income_type'].value_counts() - все хорошо
#credit['debt'].value_counts() - все хорошо
#credit['total_income'].value_counts() #- пропущенные значения
#credit['purpose'].value_counts() - одна и та же цель записана разными словам

NameError: name 'credit' is not defined

In [3]:
credit.head()

NameError: name 'credit' is not defined

### Вывод

В таблице 12 столбцов, 21525 значений.
Тип данных у Series можно разделить на три типа: int64, float64, object - целое число, дробное число и буквенное обозначение соответственно.

Наименование каждого из столбцов и их содержание:
children — количество детей в семье;
days_employed — общий трудовой стаж в днях;
dob_years — возраст клиента в годах;
education — уровень образования клиента;
education_id — идентификатор уровня образования;
family_status — семейное положение;
family_status_id — идентификатор семейного положения;
gender — пол клиента;
income_type — тип занятости;
debt — имел ли задолженность по возврату кредитов;
total_income — ежемесячный доход;
purpose — цель получения кредита.

Провверив каждый из столбцов на уникальные значения я заметил следующее:
1. В столбце 'children' есть отрицательный показатель детей и аномально-высокое количество детей у почти 80 человек;
2. В столбце 'days_employed' данные записаны через дробные числа, есть отрицательные показатели (опечатки, скорее всего) и пропущенные значения.
3. В столбце 'education' одинаковые слова записаны разным регистром, что создаст дополнительные проблемы при обработке;
4. В столбце 'family_status' также есть проблема с регистром;
5. В столбце 'gender' есть пропущенные значения;
6. В столбце 'total_income' есть пропущенные значения;
7. В столбце 'purpose' одна и та же цель получения кредита записана разными словами.

Что касается пропущенных значений, то они принадлежат у нас к типу NaN, а значит, что взаимодействовать мы с ним сможем, но это будет бессмысленно. Гораздо лучше будет заменить эти значения средним или медианным значением. Изучив таблицу я заметил то, что отсутствующие значения связаны: например, в большей мере они выявлены у пенсионеров в столбцах 'days_employed' и 'total_income'. В этом случае можно предположить, что они не указывали в анкете на получение кредита эти данные. 

На этапе предобработки необходимо будет удалить дубликаты, обработать пропущенные значения, привести к одному регистру все наименования в столбцах, изменить тип данных с вещественного на количественный, а также создать отдельные категории для столбца 'purpose'.

Для проверки наших гипотезов особое внимание стоит уделить столбцам 'debt', 'purpose', 'children', 'total_income', 'family_status'. (приоритеты я написал здесь, подумал, что будет немного неправильно выносить их еще перед самим импортом таблицы).

### Шаг 2. Предобработка данных

### Обработка пропусков

In [72]:
credit.isna().sum() #узнаем общее количество пропущенных значений и столбцы, в которых они хранятся

credit['days_employed'] = credit['days_employed'].abs() #избавляемся от отрицательных значений в столбце 'days_employed'
credit['children'] = credit['children'].abs() #отрицательное количество детей меняем на положительное

#теперь попробуем высчитать медианное значение для каждого из типов занятости
median_table = credit.groupby(['income_type']).agg({'total_income' : 'median'}) # создаем отдельную сгруппированную таблицу по типам занятости, в которой будут храниться медианы суммарного дохода

for inc_type in median_table.index: #прописываем цикл, который будем заменять пропущенные значения в столбце 'total_income' показателями, соответствующими определенной категории: пенсионер-пенсионер, предприниматель - предприниматель и тд.
    credit.loc[credit['income_type'] == inc_type, 'total_income']=credit.loc[credit['income_type'] == inc_type, 'total_income'].fillna(median_table.loc[inc_type, 'total_income'])

#сделаем то же самое и для столбца с количеством отработанных дней
med_table = credit.groupby(['income_type']).agg({'days_employed' : 'median'}) #создаем отдельную сгруппированную таблицу по типам занятости, в которой будут храниться медианы отработанных дней

for inc_type in med_table.index: #прописываем цикл, который будет заменять пропущенных значения в стоблце 'days_employed' показателями, соотвествующими определенному типу занятости
    credit.loc[credit['income_type'] == inc_type, 'days_employed']=credit.loc[credit['income_type'] == inc_type, 'days_employed'].fillna(med_table.loc[inc_type, 'days_employed'])

children_median = credit['children'].median() #посчитаем медиану для количества детей
credit['children'] = credit['children'].replace(20, children_median) #заменяем крайне высокое поличество детей медианой

credit['gender'] = credit['gender'].replace('XNA', 'M') #меняем пропущенное значение на M

#credit['gender'].value_counts() #проверям

credit.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Всего пропущенных значений в таблице: 4349.
Они находятся в столбцах: 'days_employed', 'gender' и 'total_income'.

Первоначально я избавился от отрицательных значений в столбцах'days_employed' и 'children' методом abs.

Чтобы подсчитать количество пропущенных значений я выбрал метод df.isna().sum(), т.к он наиболее подходит для этой задачи.
Для замены отсутствующих значений в стобцах 'days_empployed' и 'total_income' я выбрал медиану.
Сначала я для каждого столбца я создал отдельные таблицы, в которых хранятся их медианы с учетом типа занятости (с помощью метода аггрегирования .agg), а потом с помощью цикла заменил каждое пропущенное значение этими медианами.
Получилось довольно аккуратно.

Одно пропущенное значение в 'gender' я заменил на 'M' методом df.replace().



### Замена типа данных

In [73]:
credit['days_employed'] = credit['days_employed'].astype('int') #приводим значения к целочисленным показателям
credit['total_income'] = credit['total_income'].astype('int') #то же самое для столбца 'total_income'
credit.info() #еще раз просмотрим информацию
credit.head(50) #проверяем

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Вывод

Для преобразования данных в столбцах 'days_employed' и 'total_income' в целочисленный я использовал метод df.astype('int).
Методом pd.to_numeric я воспользовался первончально, но он переводит все значения в дробном формате, что неудобно и немного противоречит логике - количество отработанных дней не может быть дробным числом - этому я обратился к df.astype().
Он точнее и практичнее.


### Обработка дубликатов

In [74]:
#credit.duplicated().sum() #находим общее количество дубликатов

#теперь выполним ручной поиск дубликатов для строковых столбцов и сведем их к одному регистру
credit['education'] = credit['education'].str.lower()
credit['family_status'] = credit['family_status'].str.lower()
credit['income_type'] = credit['income_type'].str.lower() 
credit['purpose'] = credit['purpose'].str.lower()
#готово
#теперь удалим дубликаты с заменой индекса
credit = credit.drop_duplicates().reset_index(drop=True)

credit.duplicated().sum() #проверяем еще раз количество дубликатов

0

### Вывод

Для поиска и подсчета дубликатов был использован метод df.duplicated().dum().

После подсчета дубликатов я выполнил их ручной поиск и привел к одному регистру значения столбцов, где это было необходимо, чтобы полностью исключить возможные несостыковки при выявлении закономерностей. Для этого я использвал метод df.str.lower().

После этого, я удалил дубликаты с помощью метода df.drop_duplicates().reset_index(drop = True) с заменой индекса.

### Лемматизация

In [75]:
from pymystem3 import Mystem #импортиуем библиотеку pymystem3
m = Mystem()

credit['purpose_lemma'] = credit['purpose'].apply(m.lemmatize) #добавляем в таблицу столбец с леммами для цели кредита
#создаем функцию для выделения лемм для слов в столбце
lemmas_list = credit['purpose_lemma'].values #получаю список значений в столбце, чтобы позже поместить в один большой список all_lemmas_list
all_lemmas_list = []
for sublist in lemmas_list: #из каждого подсписка в общем списки извлекаем лемму
    for element in sublist:
        all_lemmas_list.append(element)
        
from collections import Counter #подсчитываем количество лемм с помощью метода Counter
Counter(all_lemmas_list)

#credit #проверяем, появился ли столбец и все ли работает корректно

Counter({'покупка': 5897,
         ' ': 33570,
         'жилье': 4460,
         '\n': 21454,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 478,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1314,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

### Вывод

Лемматизация была необходима для того, чтобы цели в 'purpose', которые обозначают одно и то же, но написаны разными словами, свести к нескольким общим группам. С помощью цикла в задании удалось получить сразу все леммы и их точное количество, а не выделять отдельно каждую из них, а потом уже считать.

В качестве основных лемм были выбраны следующие: "образование", "автомобиль", "свадьба", "жилье" и "недвижимость". Лемма - слово в его начальном варианте и с ними потом очень удобно взаимодействовать, написав, допустим, первую ее часть (что я и сделал дальше). Такие леммы я выбрал потому что каждое из этих слов встречается хоть в каждой строчке с целью кредита, а значит, помогут точно распределить нам эти цели на основные группы.


*Если честно, то до конца не понял, как именно здесь работает лемматизация, поэтому, если что-то не так, то подскажите, пожалуйста.




### Категоризация данных

In [76]:
def credit_purpose(purpose): #напишем функцию для замены многообразия целей кредита нашими леммами, которые получили на предыдущем этапе
    if 'жиль' in purpose or 'недвиж' in purpose:
        return 'недвижимость'
    if 'свадьб' in purpose:
        return 'свадьба'
    if 'образов' in purpose:
        return 'образование'
    if 'автомоб' in purpose:
        return 'автомобиль'
    
credit['purpose'] = credit['purpose'].apply(credit_purpose) #применяем функцию методом .apply к столбцу 'purpose'

#разделим 'total_income' на части с помощью квантилей:
credit['total_income'].quantile([.0, .2, .4, .6, .8])

#теперь разделим столбец 'total_income' на пять категорий
def income(total_income): #показатели дохода также распределяем на пять категорий
    if total_income <= 21000:
        return 'очень низкий уровень дохода'
    if 21001 <= total_income <= 50000:
        return 'низкий уровень дохода'
    if 50001 <= total_income <= 100000:
        return 'средний уровень дохода'
    if 100001 <= total_income <= 160000:
        return 'высокий уровень дохода'
    return 'очень высокий уровень дохода'

credit['category_total_income'] = credit['total_income'].apply(income) #создаем новый столбец в таблице и в него помещаем наши категории, применяю функцию def income(total_income) методом .apply()

credit #проверям, работает ли функция и появился ли новый столбец

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,category_total_income
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,"[покупка, , жилье, \n]",очень высокий уровень дохода
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,"[приобретение, , автомобиль, \n]",высокий уровень дохода
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,"[покупка, , жилье, \n]",высокий уровень дохода
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,"[дополнительный, , образование, \n]",очень высокий уровень дохода
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,"[сыграть, , свадьба, \n]",высокий уровень дохода
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,недвижимость,"[операция, , с, , жилье, \n]",очень высокий уровень дохода
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,автомобиль,"[сделка, , с, , автомобиль, \n]",высокий уровень дохода
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]",средний уровень дохода
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,автомобиль,"[на, , покупка, , свой, , автомобиль, \n]",очень высокий уровень дохода


In [77]:
credit.head(10) # выводим таблицу в последний раз, чтобы проверить все сделанные изменения и присутпить к ответу на вопросы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,category_total_income
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,"[покупка, , жилье, \n]",очень высокий уровень дохода
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,"[приобретение, , автомобиль, \n]",высокий уровень дохода
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,"[покупка, , жилье, \n]",высокий уровень дохода
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,"[дополнительный, , образование, \n]",очень высокий уровень дохода
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,"[сыграть, , свадьба, \n]",высокий уровень дохода
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость,"[покупка, , жилье, \n]",очень высокий уровень дохода
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость,"[операция, , с, , жилье, \n]",очень высокий уровень дохода
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",высокий уровень дохода
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба,"[на, , проведение, , свадьба, \n]",средний уровень дохода
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость,"[покупка, , жилье, , для, , семья, \n]",высокий уровень дохода


### Вывод

1. Я написал функцию, для использования наших лемм и сократил количество целей получения кредита до четырех: "недвижимость", "свадьба", "образование", "автомобиль";

2. Показатели заработка также распределил на четыре категории: "низкий уровень дохода", "средний уровень дохода", "высокий уровень дохода" и "очень высокий уровень дохода";


На этапе предобработки были выявлены различные проблемы в данных таблицы. Там, где что-то можно было удалить без угрозы искажения объективности, то было удалены.
Остальные данные были бережно сохранены и исправлены.
Теперь можно приступать к выявлению интересующих нас закономерностей. 


In [78]:
credit.total_income.value_counts()

142594    1070
172357     502
118514     387
150447     145
126262       3
          ... 
101387       1
138249       1
280240       1
390148       1
264193       1
Name: total_income, Length: 18608, dtype: int64

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [79]:
#делаем сводную таблицу, чтобы выявить закономерность:
#данные группирую по столбцу 'children'; в качестве интересующего значения прописываю 'debt' и методом подсчета среднего выявляю закономерность
purpose_pivot = credit.pivot_table(index = ['children'], values = 'debt', aggfunc = 'mean')
purpose_pivot #выводим таблицу

,debt
children,
0,0.075598
1,0.091658
2,0.094542
3,0.081818
4,0.097561
5,0.000000


### Вывод

Из результатов сравнения мы получили следующее:
1. Более надежны в возврате кредитов люди, у которых вообще нет детей. Это связано с: во-первых, чувством личной ответственности и, во-вторых: дети, как правило, сподвигают на непредвиденные расходы, из-за чего возврат кредита может происходить не в срок. 

1.1 Интересна ситуация с теми, у кого пять детей - полнейшее отсутствие задолженностей - тут несколько вариантов: или они всегда возвращают кредит в срок, или им просто отказывают в заеме, или сами не решаются брать. Хотя, если бы были верны два последних случая, то их не было в таблице.
2. Чаще остальных задерживают возват люди с четырьмя, двумя и одним ребенком (казалось бы, семьи с одним и двумя детьми должны возвращать кредит быстро, но нет).

3. Семьи с тремя детьми, вопреки остальным результатам, возвращают кредит быстрее, чем семьи с одним, двумя и тремя детьми (возможно, они более ответственны в этом отношении).

Что же видно из результатов исследования?

Быстрее всех с возвратом кредита справляются люди, у которых вообще нет детей. Это обусловлено повышенным чувством личной ответственности, наличием четкой цели и отсутствием дополнительных расходов на семьи и детей.


Зависимость подтверждена.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [80]:
#сводная таблица
family_status_pivot = credit.pivot_table(index = ['family_status', 'family_status_id'], values = 'debt', aggfunc = 'mean')
family_status_pivot

,,debt
family_status,family_status_id,
в разводе,3,0.071130
вдовец / вдова,2,0.065693
гражданский брак,1,0.093471
женат / замужем,0,0.075452
не женат / не замужем,4,0.097509


### Вывод

Итоги:
1. Меньше всего склонны к задолженностям по оплате кредита группы людей, попадающие в категорию "вдовец/вдова".
Это связано с тем, что люди не хотят усугублять свое финансовое положение и больше расчитывают на свои собственные средства, из-за чего оплата кредита происходит в срок. Плюс, цель кредита может быть напрямую связана с их текущим семейным положением;

2. Люди в категориях "в разводе" и "женат/замужем" практически идентичны в сроках выплаты кредита.
Так происходит, скорее всего, из-за того, что люди в разводе больше ощущают свою личную ответственность. Также на этот показатель может влиять необходимость выплаты алиментов (если есть ребенок).
Во втором случае, семейная пара вместе составляет бюджет и финансовую программу, бережнее относятся к деньгам, из-за чего невыплата кредита в срок маловероятна;

3. Больше всего вероятность невыплаты кредита в срок у категорий "гражданский брак" и "не женат/не замужем".
Здесь, как мне кажется, такие показатели из-за невысокого уровня личность ответственности и ответственности перед другими людьми.

Зависимость подтверждена.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [81]:
income_pivot = credit.pivot_table(index = ['category_total_income'], values = 'debt', aggfunc = 'mean')
income_pivot

,debt
category_total_income,
высокий уровень дохода,0.086930
низкий уровень дохода,0.059299
очень высокий уровень дохода,0.076606
очень низкий уровень дохода,1.000000
средний уровень дохода,0.080909


### Вывод

Из результатов следует следующее:
1. Самый высокий процент невозврата кредита в срок у людей с очень низким уровнем дохода. Они никогда не возвращают кредит в срок.
Это может обусловлено элементарной нехваткой денежных средств и жизнью "от зарплаты до зарплаты", плюс, сюда добавляются всевозможные займы у знакомых, и в итоге не получается вернуть займ и банку, и знакомым.

2. На втором месте люди с высоким уровнем дохода.
В случае с высокими уровнем дохода задержки возврата могут зависть от общего отношениях этих людей к кредитам. Они более свободно относятся к деньгам и им не страшно переплатить несколько процентов кредита, пусть даже и не в срок, но они будут спокойны. Не очень бережливо относятся к деньгам.

3. По середине у нас средний класс и люди с очень высоким уровнем дохода.
Средний класс представляется как люди с детьми, вследствие чего могут возникать дополнительные расходы и проблемы с распределением бюджета. Плюс некоторые пренебрежительное отношение и недостаток ответственности.
Люди с очень высоким уровнем дохода меньше двух предыдущих задерживают оплату кредита. Это может быть связано с большим чувством личной ответственности, с бережным отношением к деньгам, потому что они их ценят больше остальных, но также и с тем, что они берут кредиты на большие суммы и точно нежелают платить лишние десятки/сотни тысяч.

4. Самыми надежными заемщиками являются люди с низким уровнем дохода.
В отличии от "очень низкого уровня дохода" они более располагают денежными средствами, относятся серьезнее к каждому рублю, предпочитают на занимать лишних денег и, если берут кредит, то на определенную цель и стараются вернуть в срок.

Из выводов мы видим, что зависимость между уровнем дохода существует, но несколько в обратном порядке.
Если предположить, что точнее всех должны возвращать люди с более высоким уровнем дохода, то это точно не так.
У нас же наоборот: люди с низким уровнем дохода, а потом только с очень высоким уровнем дохода стараются возвращать кредиты в срок. А средний и высокий класс не всегда относится к этому с должным пониманием.

Зависимость подтвердилась.

- Как разные цели кредита влияют на его возврат в срок?

In [82]:
purpose_pivot = credit.pivot_table(index = ['purpose'], values = 'debt', aggfunc = ['count', 'sum', 'mean'])
purpose_pivot

,count,sum,mean
,debt,debt,debt
purpose,,,
автомобиль,4306,403,0.093590
недвижимость,10811,782,0.072334
образование,4013,370,0.092200
свадьба,2324,186,0.080034


### Вывод

Из результатов видим следующее:
1. Самый высокий процент невозврата кредита у групп с целями "автомобиль" и "образование".
В первом случаем это может быть связано с несколькими факторами:
- люди берут в кредит не самую дешевую машину, потому что еще существует стереотип, что машина должна быть дорогой, и элементарно не расчитывают свой бюджет;
- они не учитывают дополнительных и экстренных расходов на обслуживание машину: страховка, техосмотр, платная паркова, замена резины, ремонт и т.д;
Во втором случае это может быть связано с тем, что люди надеются сразу после окончания обучения или еще во время обучения начать зарабатывать большие деньги, и с них выплачить кредит. Но, как правило, немного переоценивают себя и задерживают выплаты.

2. Свадебные цели расположились по середине, но тоже с высоким процентом невыплаты.
Это, скорее всего, связано с тем, что люди хотят устроить свадьбу, как можно более масштабней и для этого берут кредит. Однако это оказывается не самым лучшим вложением, плюс добавляются дополнительные расходы, и платеж по кредиту отстрачивается.

3. Меньше всего оплату кредита задерживают люди, берущие его для оплаты или покупки недвижимости. Как правило, они занимают больше суммы и относятся к этому очень серьезно. В этом случае каждый процент значит очень большую сумму и заемщики стараются оплатить кредит в срок.

Зависимость подтверждена.

### Шаг 4. Общий вывод

Задачей исследования было определить: влияет ли семейное положение и наличие детей на возврат кредита в срок.

Исправив недочеты в таблице: дубликаты, пропуски, артефактные значения, разницу в регистрах я приступил к анализу самих данных.

Проанализировав данные таблицы, можно заключить, что наличие детей не влияет на возврат кредита в срок; как мы видим, то люди с детьми чаще остальных задерживают оплату кредита, а бездетные быстрее всех возвращают займ.
Казалось бы, должно быть наоборот, ведь дети - это дополнительная ответственность, но не только.
Дети - это еще и дополнительные и непредвиденные расходы, из-за которых деньги, отложенные на оплату кредита, могут тратиться на другие цели.

А вот семейное положение напрямую влияет на выплату кредита: чем зависимее человек от другого человека, например, брак, тем он ответственнее и выплачивает кредит в срок.
Отдельная странность касается гражданского брака, т.к на территории РФ под понимается нечто другое и было бы правильнее именовать это "сожительством".

Кроме основной задачи были поставлены дополнительные: выявить, есть ли зависимость между уровнем дохода и как разные цели кредита влияют на его возврат в срок.

Стандартная гипотеза: "чем выше уровень дохода, тем выше верятность возврата кредита в срок" не подтвердилась и все не так однозначно.
Лучше всех возвращают кредит люди с низким уровнем дохода (0.6), а хуже всех люди с очень низким уровнем дохода (1.0) и с высокими уровнем дохода (0.9), а по середине у нас средний класс и люди с очень высокими уровнем дохода.

Касательно целей кредита, то здесь зависимость прослеживается: чем больше сумма и долгосрочнее займ, как в случае с недвижимостью, тем меньше вероятность невозврата кредита в срок, потому что просрок оплаты здесь несет большой материальный ущерб для заемщика.
Высокие показатели невозврата здесь связаны с недеоцениваем заемщиками своих собственных сил и денежных средств.

Можно считать, что выявление закономерностей было произведено успешно и поставленные задачи были выполнены. 



